In [164]:
import yfinance as yf
import time

import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
import seaborn as sns

from ta.momentum import roc
from ta.momentum import RSIIndicator,rsi
from ta.trend import wma_indicator, ema_indicator
from ta.utils import dropna

import warnings
warnings.filterwarnings('ignore')

# Trading Strategy

Combination of a momentum indicator **Coppock Curve** with a mean-reversion indictor **RSI**. 

Dual-indicator approach that leverages the strengths of both the Coppock Curve and the RSI to maximize trading opportunities. This strategy is designed to harness the Coppock Curve's ability to identify long-term trend reversals and the RSI's sensitivity to short-term price movements. This combination allows traders to align themselves with the overall market momentum while pinpointing optimal entry points for trades. Ultimately, offering a more holistic view of the market and potentially filter out false signals.


## Signals
We will use Coppock Curve and RSI to each generate a value. 1 or -1. 

### Coppock curve [1,-1]
1 is generated when Coppock Curve crosses above zero. This indicates that the momentum has shifted from negative (bearish) to positive (bullish), suggesting that it might be a good time to buy.
-1 is generated when the Coppock Curve crosses below zero. This suggests that the momentum has shifted from positive (bullish) to negative (bearish), which might indicate that it might be a good time to short.

### RSI [1,-1]
40/70 We will use the values 40 to indicate oversold and 70 to indicate overbought.
1 is generated when RSI crossed above 40 from below or if RSI >50
-1 is generated when RSI crossed below 70 from above or if RSI <50

Combining the values generated by Coppock Curve and RSI
We will be able to generate the absolute value (strength of signal) denoting the confidence and direction of the signal

- **Strong Buy Signal**: (1,1) 2
- **Weak Buy Signal**: (1,-1) 0
- **Strong Sell Signal**: (-1,1) 0
- **Weak Sell Signal**: (-1,-1) -2

We will then convert the value generated to get the weights relative to the Portfolio

Legend :
2 = 3 weight  
0 = 2 weight
-2 = 1 weight

We will then use the weights given to the stock to generate the capital allocated for each stock.
Capital of Stock x = Weights of Stock x / Sum of weights of all stock

## Variations of Coppock Curve

1. **Coppock Curve with standard sets of 2 periods**: The standard Coppock Curve uses a 14-day period for the long Rate of Change (RoC) and an 11-day period for the short RoC. An alternative configuration might be to use a 21-day period for the long RoC and a 5-day period for the short RoC.

2. **Coppock Curve with different smoothing constants**: The standard Coppock Curve uses a 10-period Weighted Moving Average (WMA) for smoothing.
Alternative configurations might use a different period for the WMA.

3. The standard Coppock Curve uses a **Weighted Moving Average (WMA)** for smoothing.

In [165]:
# Strategy Logic
# Coppock Curve and RSI strategy

def get_cc_wma(df,roc1_n,roc2_n,wma_lookback):
    cc_wma = wma_indicator((roc(close=df['Close_Price'], window=roc1_n,fillna= False)) + (roc(close=df['Close_Price'], window=roc2_n,fillna= False)), window=wma_lookback,fillna= False)
    return cc_wma

def get_rsi(df,window):
    rsi_indicator = rsi(close=df['Close_Price'], window=window,fillna= False)
    return rsi_indicator

def CC_RSI_strategy(df,num):
    wma = 10
    for var in range(1,num+1):
        df[f'cc_{var}'] = get_cc_wma(df, 14, 11, wma)
        wma += 40
    df['rsi_14'] = get_rsi(df, 14)
    df = df.dropna()

    for var in range(1,num+1):
        cc_crossed_above_zero = False
        rsi_crossed_40_below = False
        rsi_crossed_70_above = False
        val = [1]
        for day in range(1,len(df)):
            # Generate strat
            cc_current = df.iloc[day][f'cc_{var}']
            cc_prev = df.iloc[day-1][f'cc_{var}']
            rsi_current = df.iloc[day]['rsi_14']
            rsi_prev = df.iloc[day-1]['rsi_14']
            if cc_crossed_above_zero:
                # check rsi
                if rsi_crossed_40_below or rsi_current > 50:
                    val.append(3)
                elif rsi_crossed_70_above or rsi_current < 50:
                    val.append(2)
                else:
                    val.append(1)

                if cc_current <= 0:
                    cc_crossed_above_zero = False
                if rsi_current < 40:
                    rsi_crossed_40_below = False
                if rsi_current > 70:
                    rsi_crossed_70_above = False
                # continue
            else:
                if rsi_crossed_40_below or rsi_current > 50:
                    val.append(2)
                elif rsi_crossed_70_above or rsi_current < 50:
                    val.append(1)
                else:
                    val.append(3)  
            # Check if CC has crossed above zero
            if cc_current > 0 and cc_prev <= 0:
                cc_crossed_above_zero = True
            else:
                cc_crossed_above_zero = False

            # Check if RSI has crossed 40 from below
            if rsi_current > 40 and rsi_prev <= 40:
                rsi_crossed_40_below = True
            else:
                rsi_crossed_40_below = False

            # Check if RSI has crossed 70 from above
            if rsi_current < 70 and rsi_prev >= 70:
                rsi_crossed_70_above= True
            else:
                rsi_crossed_70_above = False

        df[f'cc_{var}_val'] = val
    return df
    

In [166]:
class Portfolio():
    def __init__(self, stocks, capital):
        self.stocks = stocks
        self.dataframe = pd.DataFrame() #load this dataframe sequentially
        self.initial_capital = capital # this is the initial capital you want to trade with
        self.capital = capital # this capital will change depending on trades
    
    def calculate_weights(self, date):
        total_weight = 0
        weights = {}
        for stock_name, stock in self.stocks.items():
            weight = stock.data.loc[date]['cc_1_val']
            weights[stock_name] = weight
            total_weight += weight
        # Normalize weights
        for stock_name in weights:
            if total_weight != 0:
                weights[stock_name] /= total_weight
        return weights
    
    def generate_weightage(self, dates):
        '''
        dates: list of dates from start date to end date
        '''
        # Generate a dictionary of indiv stock and weights for each day
        # Normalise the weights to 100%
        for date in dates:
            weights = self.calculate_weights(date)
            # create a column in each stock.data dataframe
            for stock_name, stock in self.stocks.items():
                stock.data.loc[date, 'Weights'] = weights[stock_name]


In [150]:
class Stock():

  def __init__(self, symbol, start, end, interval, transcation_cost, strategy, verbose = True):
    self.symbol = symbol
    self.start = start
    self.end = end
    self.interval = interval
    self.transaction_cost = transcation_cost # the transaction cost for trading
    self.quantity = 0 # quantities to buy/sell
    self.position = 0 # the trades in progress, long or short
    self.trades = 0 # Number of trades
    self.verbose = verbose # if you want to see detailed output (logs)
    self.strategy = strategy # define the strategy

    self.prepare_data() # prepares the data

  def prepare_data(self):
    
    # since we are building a common class for all types of strategy, we will not calcualte the moving averages now.
    # we will calculate the returns though.
    # Since most strategies utilise close prices we are only factoring close price. However, you can alter acoordingly.

    stock_data = yf.Ticker(self.symbol)
    hist_stock = stock_data.history(start = self.start, end = self.end, interval = self.interval)
    bt_data = pd.DataFrame()
    bt_data["Close_Price"] = hist_stock["Close"]
    bt_data["Open_Price"] = hist_stock["Open"]
    bt_data["Return"] = np.log(bt_data["Close_Price"] / bt_data["Close_Price"].shift(1))
    bt_data = bt_data.dropna()
    bt_data.index = bt_data.index.date
    self.data = bt_data
    
  def close_graph(self):
    plt.figure(figsize=(15, 5))
    plt.plot(self.data["Close_Price"] ,color='black', label='Price', linestyle='dashed')
    plt.xlabel("Days")
    plt.ylabel("Price")
    plt.title("Close Prices of {}".format(self.symbol))
    plt.legend()
    plt.grid()
    plt.show()

  def return_date_price(self, bar):
    '''
    bar: is a unit of data
    '''
    # A bar is a unit of data at a given time, depends on the interval you choose, it provides you OHLCV and time info
    # Since we have modeled close prices, we will get the price and date

    date = str(self.data.index[bar])[:10] #First 10 contains the date elements, rest is time
    price = self.data.Close_Price.iloc[bar]
    return date, price

  def load_strategy(self,num):
    '''
    num: number of variations to create
    '''

    # Load the strategy to generate key columns

    self.data = self.strategy(self.data,num)

In [176]:
from matplotlib.ticker import FormatStrFormatter

class Backtester():
    
    def __init__(self, portfolio, variations):
        self.portfolio = portfolio
        self.variations = variations
        
    def check_unique_signal_count(self):
        # Validate and check the different strength across stocks
        for stock_symbol, stock in self.portfolio.stocks.items():
            print(stock_symbol)
            for i in range(1,self.variations+1):
                print(stock.data[f'cc_{i}_val'].value_counts())
            print('\n')

    def validate_stock_data(self):
        # Validate and check if stock data consistent
        data_shapes = {stock.data.shape for stock in self.portfolio.stocks.values()}
        return len(data_shapes) == 1, data_shapes.pop() if data_shapes else None

    def validate_start_dates(self):
        start_dates = set()
        # Validate and get the start date of the backtest
        for stock_symbol, stock in self.portfolio.stocks.items():
            start_dates.add(stock.data.index[0])
        return len(start_dates) == 1, start_dates.pop() if start_dates else None
        
    def validate_end_dates(self):
        end_dates = set()
        # Validate and get the end date of the backtest
        for stock_symbol, stock in self.portfolio.stocks.items():
            end_dates.add(stock.data.index[-1])
        return len(end_dates) == 1, end_dates.pop() if end_dates else None

    def verify(self):
        flag_stocks, shape = self.validate_stock_data()
        flag_start_date, start_date= self.validate_start_dates()
        flag_end_date, end_date = self.validate_end_dates()
        verify = flag_stocks and flag_start_date and flag_end_date
        self.start_bt_date = start_date
        self.end_bt_date = end_date
        return verify

    def prepare_strategy(self):
        # Code to start backtester
        stocks = self.portfolio.stocks
        for stock_name, stock in self.portfolio.stocks.items():
            stock.load_strategy(self.variations)
        if self.verify():
            first_stock_symbol = next(iter(stocks))
            dates = list(self.portfolio.stocks[first_stock_symbol].data.index)
            self.portfolio.generate_weightage(dates)
        else:
            return Exception

In [169]:
# create stock -> create portfolio -> create backtester 

stock_symbols = ['NVDA','AMZN','AAPL','PG','GOOGL']

stocks = [ Stock(
    symbol=symbol,
    start='2010-01-01',
    end='2016-12-31', 
    interval='1d', 
    transcation_cost=0, 
    strategy=CC_RSI_strategy, 
    verbose=True) 
    for symbol in stock_symbols
    ]

CC_RSI_portfolio = Portfolio(
    stocks = dict(zip(stock_symbols, stocks)),
    capital=500000
    )

backtester = Backtester(
    portfolio = CC_RSI_portfolio,
    variations = 3
    )


In [170]:
backtester.prepare_strategy()

In [171]:
backtester.check_unique_signal_count()

NVDA
cc_1_val
2    1033
1     594
3      31
Name: count, dtype: int64
cc_2_val
2    1042
1     600
3      16
Name: count, dtype: int64
cc_3_val
2    1045
1     600
3      13
Name: count, dtype: int64


AMZN
cc_1_val
2    1095
1     528
3      35
Name: count, dtype: int64
cc_2_val
2    1116
1     528
3      14
Name: count, dtype: int64
cc_3_val
2    1118
1     530
3      10
Name: count, dtype: int64


AAPL
cc_1_val
2    1058
1     568
3      32
Name: count, dtype: int64
cc_2_val
2    1065
1     577
3      16
Name: count, dtype: int64
cc_3_val
2    1074
1     574
3      10
Name: count, dtype: int64


PG
cc_1_val
2    1037
1     592
3      29
Name: count, dtype: int64
cc_2_val
2    1037
1     602
3      19
Name: count, dtype: int64
cc_3_val
2    1044
1     601
3      13
Name: count, dtype: int64


GOOGL
cc_1_val
2    1027
1     598
3      33
Name: count, dtype: int64
cc_2_val
2    1036
1     604
3      18
Name: count, dtype: int64
cc_3_val
2    1037
1     604
3      17
Name: count, dtype:

In [172]:
backtester.verify()

True

In [174]:
backtester.portfolio.stocks['GOOGL'].data.Weights.value_counts()

Weights
0.200000    405
0.222222    360
0.250000    202
0.142857    159
0.125000    151
0.166667    132
0.111111     74
0.285714     68
0.181818     43
0.333333     27
0.300000     13
0.100000     13
0.272727      9
0.375000      2
Name: count, dtype: int64

In [173]:
backtester.portfolio.stocks['AMZN'].data.Weights.value_counts()

Weights
0.200000    415
0.222222    363
0.250000    223
0.142857    133
0.125000    127
0.166667    125
0.285714     93
0.111111     69
0.181818     42
0.333333     37
0.300000     10
0.272727     10
0.100000      6
0.375000      4
0.428571      1
Name: count, dtype: int64